In [ ]:
import pandas as pd
from google.colab import drive
get_ipython().run_line_magic('matplotlib', 'inline')
import seaborn as sns
import requests
import io
from sklearn.ensemble import VotingClassifier, StackingClassifier
from sklearn.model_selection import train_test_split, cross_validate, KFold
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

drive.mount('/Drive')

Mounted at /Drive


In [ ]:
# 
#Downloading the csv file from your GitHub account

url = "https://raw.githubusercontent.com/JulienJ-44/rakuteam/main/Features/data_features_final.csv" # Make sure the url is the raw version of the file on GitHub
download = requests.get(url).content

# Reading the downloaded content and turning it into a pandas dataframe

df = pd.read_csv(io.StringIO(download.decode('utf-8')))

#Drop useless columns
df = df.drop(["Unnamed: 0.1","Unnamed: 0","imageid", "prdtypecode.1","productid", "designation","description", "best_idf"], axis=1)

# Printing out the first 5 rows of the dataframe

df.head()

# df = pd.read_csv('/Drive/My Drive/Colab/data_features_final.csv',index_col= 'productid')
# df = df.drop(columns=['Unnamed: 0','Unnamed: 0.1', 'prdtypecode.1','imageid','best_idf','designation','description'])

,productid,prdtypecode,blanc,noir,R,G,B,desc_length,desi_length,desi_nb_phrases,desc_nb_phrases,desi_nb_mots+,desc_nb_mots+,class_10,class_40,class_50,class_60,class_1140,class_1160,class_1180,class_1280,class_1281,class_1300,class_1301,class_1302,class_1320,class_1560,class_1920,class_1940,class_2060,class_2220,class_2280,class_2403,class_2462,class_2522,class_2582,class_2583,class_2585,class_2705,class_2905,desc_nb2chiffres+,desc_num,desc_poids,desc_long,desc_vol,desc_ans_mois,desc_pieces,desc_pourcent,desi_word_count,desi_char_count,desi_word_density,desi_total_length,desi_capitals,desi_num_exclamation_marks,desi_num_question_marks,desi_num_punctuation,desi_num_symbols,desi_num_unique_words,desi_words_vs_unique,desi_word_unique_percent,descri_word_count,descri_char_count,descri_word_density,descri_total_length,descri_capitals,descri_num_exclamation_marks,descri_num_question_marks,descri_num_punctuation,descri_num_symbols,descri_num_unique_words,descri_words_vs_unique,descri_word_unique_percent,desi_char_count (w/o space),descri_char_count (w/o space)
0,3804725264,10,0.896177,0.000007,251.471600,246.764924,246.580288,0,88,1,0,10,0,0.000,0.0,0.00,0.0,0.0,0.000,0.0,0.000,0.000,0.000,0.0,0.0,0.157,0.114,0.0,0.00,0.0,0.0,0.000,0.000,0.000,0.191,0.0,0.0,0.115,0.000,0.021,0,0,0,0,0,0,0,0,14,75,0.184211,88,10,0,0,1,5,11,0.785714,78.571429,1,3,0.250000,3,0,0,0,0,0,1,1.000000,100.000000,75,3
1,436067568,2280,0.804087,0.000235,233.710580,232.696360,231.392644,0,206,1,0,33,0,0.306,0.0,0.00,0.0,0.0,0.131,0.0,0.176,0.000,0.338,0.0,0.0,0.000,0.000,0.0,0.00,0.0,0.0,0.837,0.000,0.000,0.272,0.0,0.0,0.000,0.000,0.000,0,0,0,0,0,0,0,0,39,168,0.230769,206,32,0,0,1,7,34,0.871795,87.179487,1,3,0.250000,3,0,0,0,0,0,1,1.000000,100.000000,168,3
2,201115110,50,0.925060,0.000000,249.340400,251.500820,253.409132,686,76,1,3,10,72,0.000,0.1,0.48,0.0,0.0,0.000,0.0,0.000,0.274,0.000,0.0,0.0,0.144,0.000,0.0,0.26,0.0,0.0,0.000,0.644,0.657,0.212,0.0,0.0,0.000,0.000,0.020,1,0,0,0,0,0,0,0,12,65,0.181818,76,11,0,0,0,1,12,1.000000,100.000000,109,652,0.166922,760,61,0,0,10,0,88,0.807339,80.733945,65,652
3,50418756,1280,0.476548,0.000221,169.924608,152.227476,149.065240,0,63,1,0,7,0,0.000,0.0,0.00,0.0,0.0,0.000,0.0,0.222,0.000,0.000,0.0,0.0,0.000,0.000,0.0,0.00,0.0,0.0,0.000,0.000,0.000,0.000,0.0,0.0,0.000,0.000,0.000,0,0,0,0,0,0,0,0,10,54,0.181818,63,7,0,0,0,2,9,0.900000,90.000000,1,3,0.250000,3,0,0,0,0,0,1,1.000000,100.000000,54,3
4,278535884,2705,0.781337,0.003199,225.403116,234.072128,238.177512,187,20,1,3,4,22,0.000,0.0,0.00,0.0,0.0,0.000,0.0,0.000,0.000,0.000,0.0,0.0,0.000,0.000,0.0,0.00,0.0,0.0,0.000,0.000,0.000,0.000,0.0,0.0,0.000,0.111,0.000,0,0,0,0,0,0,0,0,4,17,0.222222,20,4,0,0,0,0,4,1.000000,100.000000,34,180,0.187845,213,4,0,0,9,4,30,0.882353,88.235294,17,180


In [ ]:
# df.info()

In [ ]:

df['class'] = df['prdtypecode']
df = df.drop('prdtypecode', axis=1)
print(df.head())
# ​
df = df.replace({'class': {10: 1, 2280:2,   50:3, 1280:4, 2705:5, 2522:6, 2582:7, 1560:8, 1281:9, 1920:10, 2403:11,
       1140:12, 2583:13, 1180:14, 1300:15, 2462:16, 1160:17, 2060:18,   40:19,   60:20, 1320:21, 1302:22,
       2220:23, 2905:24, 2585:25, 1940:26, 1301:0}})

target = df['class']

features = df.drop(['class'], axis=1)

    productid     blanc  ...  descri_char_count (w/o space)  class
0  3804725264  0.896177  ...                              3     10
1   436067568  0.804087  ...                              3   2280
2   201115110  0.925060  ...                            652     50
3    50418756  0.476548  ...                              3   1280
4   278535884  0.781337  ...                            180   2705

[5 rows x 74 columns]


In [ ]:
#Prepare trainning and test data
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.2, random_state = 123)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X_train)

X_train  = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
from time import time
t0 = time()
# {}
clf1 = RandomForestClassifier(max_features ='sqrt', n_jobs = -1,n_estimators = 500)
clf1.fit(X_train, y_train)

y_pred = clf1.predict(X_test)
dfcross=pd.crosstab(y_test.ravel(), y_pred, rownames=['Realité'], colnames=['Prédiction'], normalize=1)

print('Processing time: {} mins'.format(round((time() - t0) / 60, 2)))

classif = classification_report(y_test, y_pred)
print(classif)



Processing time: 2.52 mins
              precision    recall  f1-score   support

           0       0.94      0.67      0.78       154
           1       0.47      0.59      0.53       601
           2       0.71      0.79      0.75       964
           3       0.75      0.62      0.68       341
           4       0.61      0.66      0.63       944
           5       0.62      0.81      0.71       536
           6       0.68      0.75      0.71       993
           7       0.76      0.56      0.64       524
           8       0.61      0.76      0.68       967
           9       0.55      0.35      0.43       415
          10       0.90      0.86      0.88       866
          11       0.69      0.75      0.72       979
          12       0.68      0.64      0.66       552
          13       0.85      0.96      0.90      2045
          14       0.78      0.45      0.57       139
          15       0.92      0.88      0.90      1006
          16       0.78      0.72      0.75       298


In [ ]:
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score

params_clf1 = {'n_estimators': [ 250, 500, 1000],
                'min_samples_leaf': [1, 3, 5],
                'max_features': ['sqrt', 'log2']}

gridcv = GridSearchCV(clf1, param_grid=  params_clf1, scoring= 'accuracy', cv = 3)

In [ ]:

t0 = time()

gridcv.fit(X_train, y_train)


pd.DataFrame(gridcv.cv_results_)[['params', 'mean_test_score' , 'std_test_score']]

print('Processing time: {} mins'.format(round((time() - t0) / 60, 2)))

Processing time: 96.49 mins


In [ ]:
ypred_proba = clf1.predict_proba(X_test)

In [ ]:
df_tosave=pd.DataFrame(ypred_proba)
path = '/Drive/My Drive/Colab'
df_tosave.to_csv(f'{path}/ypred_proba_RandomForest_Global_score0_74.csv')